In [1]:
import datetime

import pandas as pd
import tensorflow as tf
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from tensorflow import keras

C:\Users\mpich\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\mpich\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\mpich\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\mpich\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:529: FutureWarning: Passi

In [2]:
df = pd.read_csv("manipulated_pun.csv", sep=";", decimal=',', header='infer')
df.head()

,Data,Ora,PUN
0,20170101,1,53.30
1,20170101,2,52.00
2,20170101,3,51.00
3,20170101,4,47.27
4,20170101,5,45.49


In [3]:
df["Data"] = df["Data"].apply(lambda x: str(x))
df["Data"] = df["Data"].apply(lambda x: datetime.datetime.strptime(x,"%Y%m%d"))
df = df.loc[df['Data'] <= '2020-01-01']

In [4]:
df["Ora2"] = pd.to_datetime(df.Ora, unit="h").dt.strftime("%H:%M")
df["Datetime"] = df["Data"].dt.strftime("%Y-%m-%d") + " " + df["Ora2"]
df.index = pd.DatetimeIndex(df.Datetime)

In [5]:
df.tail(30)

,Data,Ora,PUN,Ora2,Datetime
Datetime,,,,,
2019-12-31 19:00:00,2019-12-31,19,59.28,19:00,2019-12-31 19:00
2019-12-31 20:00:00,2019-12-31,20,56.48,20:00,2019-12-31 20:00
2019-12-31 21:00:00,2019-12-31,21,53.24,21:00,2019-12-31 21:00
2019-12-31 22:00:00,2019-12-31,22,50.68,22:00,2019-12-31 22:00
2019-12-31 23:00:00,2019-12-31,23,48.61,23:00,2019-12-31 23:00
2019-12-31 00:00:00,2019-12-31,24,42.30,00:00,2019-12-31 00:00
2020-01-01 01:00:00,2020-01-01,1,41.88,01:00,2020-01-01 01:00
2020-01-01 02:00:00,2020-01-01,2,38.64,02:00,2020-01-01 02:00
2020-01-01 03:00:00,2020-01-01,3,36.55,03:00,2020-01-01 03:00


In [6]:
target = df["PUN"]
df["df24"] = target.shift(24)

In [7]:
df

,Data,Ora,PUN,Ora2,Datetime,df24
Datetime,,,,,,
2017-01-01 01:00:00,2017-01-01,1,53.30,01:00,2017-01-01 01:00,NaN
2017-01-01 02:00:00,2017-01-01,2,52.00,02:00,2017-01-01 02:00,NaN
2017-01-01 03:00:00,2017-01-01,3,51.00,03:00,2017-01-01 03:00,NaN
2017-01-01 04:00:00,2017-01-01,4,47.27,04:00,2017-01-01 04:00,NaN
2017-01-01 05:00:00,2017-01-01,5,45.49,05:00,2017-01-01 05:00,NaN
...,...,...,...,...,...,...
2020-01-01 20:00:00,2020-01-01,20,48.63,20:00,2020-01-01 20:00,56.48
2020-01-01 21:00:00,2020-01-01,21,48.96,21:00,2020-01-01 21:00,53.24
2020-01-01 22:00:00,2020-01-01,22,43.30,22:00,2020-01-01 22:00,50.68


In [8]:
df.dropna(inplace = True)
mask_test = df.index > "2019-12-25"
mask_train = df.index <= "2019-12-25"

test = df.loc[mask_test]
train = df.loc[mask_train]

In [9]:
trainX = train['df24'][24:].values
testX = test['df24'].values
trainY = train['PUN'][24:].values
testY = test["PUN"].values

trainX.shape = ( 26065, 1)
testX.shape = ( 191, 1) 
trainY.shape = ( 26065 , 1)
testY.shape = ( 191, 1)

ValueError: cannot reshape array of size 191 into shape (167,1)

In [ ]:
# NNR works better with scaled data
scaler = MinMaxScaler(feature_range=(0,1))
trainX = scaler.fit_transform(trainX)
trainY = scaler.fit_transform(trainY)
testX = scaler.fit_transform(testX)

In [ ]:
model = tf.keras.Sequential()

model.add(keras.layers.Dense(100, activation='relu', input_shape=(trainX.shape[1],)))
model.add(keras.layers.Dense(1024, activation='relu'))
model.add(keras.layers.Dense(1))
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=4,
                                                  verbose=0, mode='auto')
model.fit(trainX, trainY, epochs=100, batch_size=100, callbacks=[early_stopping])

In [ ]:
predicted = model.predict(testX)
predicted = scaler.inverse_transform(predicted)
predicted_ = pd.DataFrame(predicted)
predicted_.head()

In [ ]:
testY = pd.DataFrame(testY)
testY = testY.reset_index()
results_vect = pd.concat([testY, predicted_], axis=1)
results_vect.columns = ['Datetime', 'Target', 'Reg']
results_vect 

In [ ]:
predicted_final = predicted_.shift(-24)
results_vect = pd.concat([testY, predicted_final], axis=1)
results_vect.columns = ['Datetime', 'Target', 'predicted_final']

In [ ]:
results_vect.dropna(inplace=True)
results_vect[['Target', 'predicted_final']].plot()

In [ ]:
# computing MAE value
results_vect['diff'] = results_vect['Target'] - results_vect['predicted_final']
results_vect['diff'] = results_vect['diff'].apply(lambda x: abs(x))
results_vect['diff'].mean()

In [ ]:
# computing MAPE value
results_vect['diff2'] = results_vect['diff']/results_vect['Target']
results_vect['diff2'].mean()

## Checking Performances on Training Set

In [ ]:
predicted2 = model.predict(trainX)
predicted2 = scaler.inverse_transform(predicted2)
predicted_2 = pd.DataFrame(predicted2)
predicted_2.head()

In [ ]:
results_vect2 = pd.concat([trainY, predicted_final2], axis=1)
results_vect2.columns = ['Datetime', 'Target', 'predicted_final']

In [ ]:
# %%
trainY = train['PUN'][168:].values
trainY.shape = ( 25944, 1)
trainY= pd.DataFrame(trainY)
trainY = trainY.reset_index()

In [ ]:
predicted_final2 = predicted_2.shift(-24)
len(predicted_final2)

In [ ]:
results_vect2 = pd.concat([trainY, predicted_2], axis=1)
results_vect2.columns = ['Datetime', 'Target', 'Reg']
results_vect2 

In [ ]:
results_vect2.dropna(inplace=True)
results_vect2[['Target', 'predicted_final']].plot()

In [ ]:
# computing MAE value
results_vect2['diff'] = results_vect2['Target'] - results_vect2['predicted_final']
results_vect2['diff'] = results_vect2['diff'].apply(lambda x: abs(x))
results_vect2['diff'].mean()

In [ ]:
# computing MAPE value
results_vect2['diff2'] = results_vect2['diff']/results_vect2['Target']
results_vect2['diff2'].mean()